# Import Libary Package

In [3]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.3 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=1e90264c6e92bf47e5fa457d07ed750de3cf7f6e9cbade2227a77785e6b5b08a
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [4]:
!pip install emot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.2 MB/s eta 0:00:00


In [5]:
!pip install levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.4 MB/s eta 0:00:00


In [6]:
!pip install tqdm

In [7]:
import pandas as pd
import numpy as np
import json
import re
from difflib import SequenceMatcher
import itertools
from itertools import combinations, product, chain
from pprint import pprint
from ast import literal_eval
from tqdm import tqdm

from emot.emo_unicode import UNICODE_EMOJI
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from Levenshtein import ratio

from __future__ import unicode_literals
import operator
import sys

try:
    from html.parser import HTMLParser
except ImportError:
    from HTMLParser import HTMLParser

try:
    import html
except ImportError:
    pass
import py_vncorenlp

# Load data

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/raw_data/train.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/raw_data/test.csv').drop('Unnamed: 0', axis=1)
dev = pd.read_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/raw_data/dev.csv').drop('Unnamed: 0', axis=1)
print(train.shape, test.shape, dev.shape)

(8372, 2) (1045, 2) (1050, 2)


In [ ]:
ViLexNorm = pd.concat([train, test, dev]).reset_index(drop=True)

In [ ]:
ViLexNorm

,original,normalized
0,thích anh cá mập k,thích anh cá mập không
1,cứ ngây thơ thế thoai :)),cứ ngây thơ thế thôi :))
2,bà Nghê xinh vậy mà t thấy k bằng bà ChiPu luô...,bà Nghê xinh vậy mà tôi thấy không bằng bà Chi...
3,Ê k khóc được làm thế nào má =))?,Ê không khóc được làm thế nào má =))?
4,Có biến gì hong dẫy :)),Có biến gì không vậy :))
...,...,...
10462,Từ lúc đu idol Hàn và diễn viên Thái thì hầu n...,Từ lúc đu idol Hàn và diễn viên Thái thì hầu n...
10463,A hay nói e á,Anh hay nói em á
10464,Đọc jd mà nó dễ quá đâm ra sợ cty lừa :))),Đọc jd mà nó dễ quá đâm ra sợ công ty lừa :)))
10465,Chỗ còn tuyển hong rủ tui vô làm nói chuyện dí...,Chỗ còn tuyển không rủ tui vô làm nói chuyện v...


# Tokenization Fuction

In [ ]:
def regex_or(*items):
    return '(?:' + '|'.join(items) + ')'

In [ ]:
punctChars = r"['\"“”‘’.?!…,:;]"
#punctSeq   = punctChars+"+"	#'anthem'. => ' anthem '.
punctSeq   = r"['\"“”‘’]+|[.?!,…]+|[:;]+"	#'anthem'. => ' anthem ' .
entity     = r"&(?:amp|lt|gt|quot);"

#URLs
urlStart1  = r"(?:https?://|\bwww\.)"
commonTLDs = r"(?:com|org|edu|gov|net|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|pro|tel|travel|xxx)"
ccTLDs	 = r"(?:ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|" + \
r"bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|" + \
r"er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|" + \
r"hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|" + \
r"lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|" + \
r"nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|sk|" + \
r"sl|sm|sn|so|sr|ss|st|su|sv|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|" + \
r"va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|za|zm|zw)"	#TODO: remove obscure country domains?
urlStart2  = r"\b(?:[A-Za-z\d-])+(?:\.[A-Za-z0-9]+){0,3}\." + regex_or(commonTLDs, ccTLDs) + r"(?:\."+ccTLDs+r")?(?=\W|$)"
urlBody    = r"(?:[^\.\s<>][^\s<>]*?)?"
urlExtraCrapBeforeEnd = regex_or(punctChars, entity) + "+?"
urlEnd     = r"(?:\.\.+|[<>]|\s|$)"
url        = regex_or(urlStart1, urlStart2) + urlBody + "(?=(?:"+urlExtraCrapBeforeEnd+")?"+urlEnd+")"

# Numeric
timeLike   = r"\d+(?::\d+){1,2}"  # (1:00:00)
numdotNum = r"\d+[.,]\d+"
rationum = r"\d+\-\d+"
numberWithCommas = r"(?:(?<!\d)\d{1,3},)+?\d{3}" + r"(?=(?:[^,\d]|$))"  #(1,5  2,5)
number_slash_number = r'-?\d+[,\.]\d+/\d+([,\.]\d+)?'
#currency
numComb	 = u"[\u0024\u058f\u060b\u09f2\u09f3\u09fb\u0af1\u0bf9\u0e3f\u17db\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6\u00a2-\u00a5\u20a0-\u20b9]?\\d+(?:\\.\\d+)+%?"

boundaryNotDot = regex_or("$", r"\s", r"[“\"?!,:;]", entity)
separators  = "(?:--+|―|—|~|–|=)"
decorations = u"(?:[♫♪]+|[★☆]+|[♥❤♡]+|[\u2639-\u263b]+|[\ue001-\uebbb]+)"
thingsThatSplitWords = r"[^\s\.,?\"]"


#  Emoticons
# myleott: in Python the (?iu) flags affect the whole expression
#normalEyes = "(?iu)[:=]" # 8 and x are eyes but cause problems
normalEyes = "[:=+-]" # 8 and x are eyes but cause problems
wink = "[;]"
noseArea = "(?:|-|[^a-zA-Z0-9 ]|\s)" # doesn't get :'-(

happyMouths = r"[D\)\]\}\>]+"
sadMouths = r"[\(\[\{\<]+"
tongue = "[pPdđĐ3*]+"
otherMouths = r"(?:[oO]+|[/\\]+|[vV]+|[Ss]+|[|]+)" # remove forward slash if http://'s aren't cleaned
bfLeft = u"(♥|0|[oO]|°|[vV]|\\$|[tT]|[xX]|;|\u0ca0|@|ʘ|•|・|◕|\^|¬|\\*)"
bfCenter = r"(?:[\.]|•|・x・|・|[_-]+)"
bfRight = r"\2"
s3 = r"(?:--['\"])"
s4 = r"(?:<|&lt;|>|&gt;)[\._-]+(?:<|&lt;|>|&gt;)"
s5 = "(?:[.][_]+[.])"

double_eyes = r"\^\^|@@"
basicface = "(?:" +bfLeft+bfCenter+bfRight+ ")|" +s3+ "|" +s4+ "|" + s5
eye_with_nose = "(?:" +bfLeft+bfCenter+bfLeft+ ")" # Like ^.^, T_T,

eeLeft = r"[＼\\ƪԄ\(（<>;ヽ\-=~\*]+"
eeRight= u"[\\-=\\);'\u0022<>ʃ）/／ノﾉ丿╯σっµ~\\*]+"
eeSymbol = r"[^A-Za-z0-9\s\(\)\*:=-]"
eastEmote = eeLeft + "(?:"+basicface+"|" +eeSymbol+")+" + eeRight
oOEmote = r"(?:[oO]" + bfCenter + r"[oO])"

Hearts = "(?:<+/?3+)+"
Arrows = regex_or(r"(?:<*[-―—=]*>+|<+[-―—=]*>*)", u"[\u2190-\u21ff]+")
Hashtag = "#[a-zA-Z0-9_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ]+"
AtMention = "[@＠][a-zA-Z0-9_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ]+"
Bound = r"(?:\W|^|$)"
Email = regex_or("(?<=(?:\W))", "(?<=(?:^))") + r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4}(?=" +Bound+")"

# Edge punctuation
edgePunctChars    = u"'\"“”‘’«»{}\\(\\)\\[\\]\\*&" #add \\p{So}? (symbols)
edgePunct    = "[" + edgePunctChars + "]"
notEdgePunct = "[a-zA-Z0-9_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ]" # content characters
offEdge = r"(^|$|:|;|\s|\.|,)"  # colon here gets "(hello):" ==> "( hello ):"
EdgePunctLeft  = re.compile(offEdge + "("+edgePunct+"+)("+notEdgePunct+")", re.UNICODE)
EdgePunctRight = re.compile("("+notEdgePunct+")("+edgePunct+"+)" + offEdge, re.UNICODE)

In [ ]:
emoticon = regex_or(
        # Standard version  :) :( :] :D :P
        "(?:>|&gt;)?" + regex_or(normalEyes, wink) + regex_or(noseArea,"[Oo]", "\*") + regex_or(tongue+r"(?=\W|$|RT|rt|Rt)", otherMouths+r"(?=\W|$|RT|rt|Rt)", sadMouths, happyMouths),

        # reversed version (: D:  use positive lookbehind to remove "(word):"
        # because eyes on the right side is more ambiguous with the standard usage of : ;
        regex_or("(?<=(?: ))", "(?<=(?:^))") + regex_or(sadMouths,happyMouths,otherMouths) + noseArea + regex_or(normalEyes, wink) + "(?:<|&lt;)?",

        #inspired by http://en.wikipedia.org/wiki/User:Scapler/emoticons#East_Asian_style
        eastEmote.replace("2", "1", 1), basicface,eye_with_nose,
        # iOS 'emoji' characters (some smileys, some symbols) [\ue001-\uebbb]
        # TODO should try a big precompiled lexicon from Wikipedia, Dan Ramage told me (BTO) he does this

        # myleott: o.O and O.o are two of the biggest sources of differences
        #          between this and the Java version. One little hack won't hurt...
        oOEmote, double_eyes
)

In [ ]:
# m/d/yyyy + mm/dd/yyyy
d1 = r"^(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])/(?:[0-9]{2})?[0-9]{2}$"
# d/m/yyyy + dd/mm/yyyy
d2 = r"^(3[01]|[12][0-9]|0?[1-9])/(1[0-2]|0?[1-9])/(?:[0-9]{2})?[0-9]{2}$"
# m/d or mm/dd
d3 = r"^(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])"
# d/m or dd/mm
d4 = r"^(3[01]|[12][0-9]|0?[1-9])/(1[0-2]|0?[1-9])"

date = regex_or(d1,d2,d3,d4)

In [ ]:
# We will be tokenizing using these regexps as delimiters
# Additionally, these things are "protected", meaning they shouldn't be further split themselves.
Protected  = re.compile(
    regex_or(
        Hearts,
        url,
        Email,
        timeLike,
        date,
        numdotNum,
        rationum,
        numberWithCommas,
        number_slash_number,
        numComb,
        emoticon,
        Arrows,
        entity,
        punctSeq,
        separators,
        decorations,
        Hashtag,
        AtMention), re.UNICODE)

In [ ]:
emoji_list = list(UNICODE_EMOJI)

In [ ]:
def splitEdgePunct(input):
    input = EdgePunctLeft.sub(r"\1\2 \3", input)
    input = EdgePunctRight.sub(r"\1 \2\3", input)
    return input

def addAllnonempty(master, smaller):
    for s in smaller:
        strim = s.strip()
        if (len(strim) > 0):
            master.append(strim)
    return master

def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

def split_emoji_text(text):
    result = ''.join((' {} '.format(el) if el in emoji_list else el for el in text))
    return result


def split_emoji_emoji(textArray):
    result = []
    for word in textArray:
        is_emoji = False

        # iterate element of characters and check char is emoji if yes
        # then append into result list and set is_emogi flag true.
        for char in word:
            if char in [*emoji_list]:
                result.append(char)
                is_emoji = True

        # check is_emoji flag is false then append string into list.
        if not is_emoji:
            result.append(word)

    return result

def remove_dotNotend_in_list(lst):
  for i, e in enumerate(lst):
    if e == '.':
      if (i != len(lst) -1) and (i == 0 or not lst[i + 1].isupper()):
        lst[i] = ''
      else:
        pass
  return lst

def wordSegmentandKeepicon(text):

    # Do the no-brainers first
    splitPunctText = splitEdgePunct(text)

    textLength = len(splitPunctText)

    # BTO: the logic here got quite convoluted via the Scala porting detour
    # It would be good to switch back to a nice simple procedural style like in the Python version
    # ... Scala is such a pain.  Never again.

    # Find the matches for subsequences that should be protected,
    # e.g. URLs, 1.0, U.N.K.L.E., 12:53
    bads = []
    badSpans = []
    for match in Protected.finditer(splitPunctText):
        # The spans of the "bads" should not be split.
        if (match.start() != match.end()): #unnecessary?
            bads.append( [splitPunctText[match.start():match.end()]] )
            badSpans.append( (match.start(), match.end()) )
    #print(bads)
    #print(badSpans)
    # Create a list of indices to create the "goods", which can be
    # split. We are taking "bad" spans like
    #     List((2,5), (8,10))
    # to create
    #     List(0, 2, 5, 8, 10, 12)
    # where, e.g., "12" here would be the textLength
    # has an even length and no indices are the same
    indices = [0]
    for (first, second) in badSpans:
        indices.append(first)
        indices.append(second)
    indices.append(textLength)
    #print(indices)
    # Group the indices and map them to their respective portion of the string
    splitGoods = []
    for i in range(0, len(indices), 2):
        goodstr = splitPunctText[indices[i]:indices[i+1]]
        goodstr = rdrsegmenter.word_segment(goodstr)
        #splitstr = goodstr.strip().split(" ")
        splitGoods.append(goodstr)

    #  Reinterpolate the 'good' and 'bad' Lists, ensuring that
    #  additonal tokens from last good item get included
    zippedStr = []
    for i in range(len(bads)):
        zippedStr = addAllnonempty(zippedStr, splitGoods[i])
        zippedStr = addAllnonempty(zippedStr, bads[i])
    zippedStr = addAllnonempty(zippedStr, splitGoods[len(bads)])

    # BTO: our POS tagger wants "ur" and "you're" to both be one token.
    # Uncomment to get "you 're"
    #splitStr = []
    #for tok in zippedStr:
    #    splitStr.extend(splitToken(tok))
    #zippedStr = splitStr

    return zippedStr

def simpleTokenize(text):

    # Do the no-brainers first
    splitPunctText = splitEdgePunct(text)

    textLength = len(splitPunctText)

    # BTO: the logic here got quite convoluted via the Scala porting detour
    # It would be good to switch back to a nice simple procedural style like in the Python version
    # ... Scala is such a pain.  Never again.

    # Find the matches for subsequences that should be protected,
    # e.g. URLs, 1.0, U.N.K.L.E., 12:53
    bads = []
    badSpans = []
    for match in Protected.finditer(splitPunctText):
        # The spans of the "bads" should not be split.
        if (match.start() != match.end()): #unnecessary?
            bads.append( [splitPunctText[match.start():match.end()]] )
            badSpans.append( (match.start(), match.end()) )

    # Create a list of indices to create the "goods", which can be
    # split. We are taking "bad" spans like
    #     List((2,5), (8,10))
    # to create
    #     List(0, 2, 5, 8, 10, 12)
    # where, e.g., "12" here would be the textLength
    # has an even length and no indices are the same
    indices = [0]
    for (first, second) in badSpans:
        indices.append(first)
        indices.append(second)
    indices.append(textLength)

    # Group the indices and map them to their respective portion of the string
    splitGoods = []
    for i in range(0, len(indices), 2):
        goodstr = splitPunctText[indices[i]:indices[i+1]]
        splitstr = goodstr.strip().split(" ")
        splitGoods.append(splitstr)

    #  Reinterpolate the 'good' and 'bad' Lists, ensuring that
    #  additonal tokens from last good item get included
    zippedStr = []
    for i in range(len(bads)):
        zippedStr = addAllnonempty(zippedStr, splitGoods[i])
        zippedStr = addAllnonempty(zippedStr, bads[i])
    zippedStr = addAllnonempty(zippedStr, splitGoods[len(bads)])

    # BTO: our POS tagger wants "ur" and "you're" to both be one token.
    # Uncomment to get "you 're"
    #splitStr = []
    #for tok in zippedStr:
    #    splitStr.extend(splitToken(tok))
    #zippedStr = splitStr

    return zippedStr

def preprocess_with_word_segment(text):
    #text = text.lower()
    text = split_emoji_text(text)
    text = wordSegmentandKeepicon(text)
    text = remove_dotNotend_in_list(text)
    text = split_emoji_emoji(text)
    text = list(filter(str.strip, text))
    sentence = ' '.join(text)
    sentence = sentence.lower()
    return sentence

def preprocess(text):
    #text = text.lower()
    text = split_emoji_text(text)
    text = simpleTokenize(text)
    text = remove_dotNotend_in_list(text)
    text = split_emoji_emoji(text)
    text = list(filter(str.strip, text))
    sentence = ' '.join(text)
    sentence = sentence.lower()
    return sentence

# Word Segmentation

In [ ]:
# Automatically download VnCoreNLP components from the original repository
# and save them in some local machine folder
py_vncorenlp.download_model(save_dir='/content')

In [ ]:
# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

In [ ]:
text = 'con mẹ nó đoạn hôm qua t làm rồi nhưng mà nay lỗi'
output = preprocess_with_word_segment(text)
print("Orignal:", text)
print("Segmented:", output)

Orignal: con mẹ nó đoạn hôm qua t làm rồi nhưng mà nay lỗi
Segmented: con_mẹ nó đoạn hôm_qua t làm rồi nhưng_mà nay lỗi


In [ ]:
text = ViLexNorm['original'][3964]
output = preprocess_with_word_segment(text)
print("Orignal:")
print("Text:", text)
print("Segmented:", output)

text = ViLexNorm['normalized'][3964]
output = preprocess_with_word_segment(text)
print("Normalized:")
print("Text:", text)
print("Segmented:", output)

Orignal:
Text: cô lại đây mà xem, tiểu bảo bối còn quá nhỏ v mà
Segmented: cô lại đây mà xem , tiểu bảo_bối còn quá nhỏ v mà
Normalized:
Text: cô lại đây mà xem, tiểu bảo bối còn quá nhỏ vậy mà
Segmented: cô lại đây mà xem , tiểu bảo_bối còn quá nhỏ vậy_mà


In [ ]:
print(ViLexNorm['original'][8787])
print(ViLexNorm['normalized'][8787])

Ăn không có ngon k mn? Có chua k?
Ăn có không ngon không mọi người? Có chua không?


# Alignment Function

In [ ]:
def generate_aligned_index(mutual_ids, lst_len):
  delta = [int(not((mutual_ids[i] == 0) and (mutual_ids[i] == mutual_ids[i-1]))) for i in range(1, lst_len)]
  aligned_index = [0]
  for value in delta:
    aligned_index.append(aligned_index[-1] + value)
  return aligned_index

In [ ]:
def align_tokens(source_list_aligned_idx, source_list):
  result_dict = {}
  for idx, value in zip(source_list_aligned_idx, source_list):
    if idx in result_dict:
        result_dict[idx].append(value)
    else:
        result_dict[idx] = [value]

  # Convert the dictionary values to lists if needed
  result_lists = list(result_dict.values())
  return result_lists

def match_idx(list1, list2):
  indices_list1 = [index for index, value in enumerate(list1) if value == 1]
  indices_list2 = [index for index, value in enumerate(list2) if value == 1]
  return indices_list1, indices_list2

def find_missing_intervals(mutual_ids):
  intervals = []
  current_interval = []

  for i, value in enumerate(mutual_ids):
    if value == 0:
        current_interval.append(i)
    elif value == 1 and current_interval:
        intervals.append(current_interval)
        current_interval = []

  # Check if there's any remaining interval at the end
  if current_interval:
    intervals.append(current_interval)
  return intervals

In [ ]:
def find_all_idx(lst, value):
  idx_lst = []
  for i in range(len(lst)):
    if lst[i] == value:
      idx_lst.append(i)
  return idx_lst

In [ ]:
def realign(list_1, list_2):
  new_src_align_idx = [-1]*len(list_1)
  new_tgt_align_idx = [-1]*len(list_2)
  lst_1 = list_1
  lst_2 = list_2
  align_idx = -1
  sim_lst = []
  max_align_lst_len = 3
  while len(lst_1) != 0:
    original_lst_2 = lst_2
    lst_2 = lst_2[:max_align_lst_len]
    for token in lst_2:
      #sim = similar(run_strip_accents(token), lst_1[0])
      if '_' in token:
        syllables = token.split("_")
        sub_sim_lst = []
        for syllable in syllables:
          sub_sim_lst.append(ratio(syllable, lst_1[0]))
        sim = max(sub_sim_lst)
      else:
        sim = ratio(token, lst_1[0])
      sim_lst.append(sim)
    max_index = sim_lst.index(max(sim_lst))
    tgt_token = lst_2[max_index]
    tgt_index = list_2.index(tgt_token)
    src_index = list_1.index(lst_1[0])
    if new_tgt_align_idx[tgt_index] == -1:
      align_idx += 1
      new_tgt_align_idx[tgt_index] = align_idx
    else:
      new_tgt_align_idx[tgt_index] = align_idx
    new_src_align_idx[src_index] = align_idx
    lst_1 = lst_1[1:]
    lst_2 = original_lst_2[max_index:]
    sim_lst = []

  if -1 in new_tgt_align_idx:
    index_lst = find_all_idx(new_tgt_align_idx, -1)
    for i in index_lst:
      sim_lst = []
      for token in list_1:
        sim = ratio(token, list_2[i])
        sim_lst.append(sim)
      max_sim = max(sim_lst)
      max_indices = find_all_idx(sim_lst, max_sim)
      if len(max_indices) == 0:
        align_index = new_src_align_idx[max_indices[0]]
      else:
        idx_dis = [abs(idx - 3) for idx in max_indices]
        min_idx_dis = min(idx_dis)
        idx = idx_dis.index(min_idx_dis)
        align_index = new_src_align_idx[max_indices[idx]]
      new_tgt_align_idx[i] = align_index
  return new_src_align_idx, new_tgt_align_idx

In [ ]:
def check_length_not_equal_to_one(list1, list2, i):
    if len(list1) > i and len(list2) > i:
        return len(list1[i]) != 1 and len(list2[i]) != 1
    return False

def extract_dissimilar_words(list1, list2):
    result_list1 = []
    result_list2 = []

    for sub_list1, sub_list2 in zip(list1, list2):
      new_sub_list1 = [token for token in sub_list1 if token not in sub_list2]
      new_sub_list2 = [token for token in sub_list2 if token not in sub_list1]
      if new_sub_list1 and new_sub_list2:
        result_list1.append(new_sub_list1)
        result_list2.append(new_sub_list2)
    return result_list1, result_list2

def filter_by_aligned_idx(lst, aligned_idx_lst, aligned_idx):
  idx_lst = find_all_idx(aligned_idx_lst, aligned_idx)
  filtered_lst = []
  for i in idx_lst:
    filtered_lst.append(lst[i])
  return filtered_lst

def print_align_res(lst_1, lst_2, align_idx_1, align_idx_2):
  align_indices = set(align_idx_1)
  src, tgt = [], []
  for i in align_indices:
    noisy = filter_by_aligned_idx(lst_1, align_idx_1, i)
    gold = filter_by_aligned_idx(lst_2, align_idx_2, i)
    src.append(noisy)
    tgt.append(gold)
  return src, tgt

def check_realign(list1, list2):
  src_sentence, tgt_sentence = [], []
  for sub_list1, sub_list2 in zip(list1, list2):
      new_src_align_idx, new_tgt_align_idx  = realign(sub_list1, sub_list2)
      new_src_align_token, new_tgt_align_token = print_align_res(sub_list1, sub_list2, new_src_align_idx, new_tgt_align_idx)
      for j in range(max(len(new_src_align_token), len(new_tgt_align_token))):
        src_sentence.append(new_src_align_token[j])
        tgt_sentence.append(new_tgt_align_token[j])
  return src_sentence, tgt_sentence

In [ ]:
def align(source, target, segment=False):
  if segment:
    source = preprocess_with_word_segment(source) #sentence: "X Y Z ..."
    target = preprocess_with_word_segment(target)
  else:
    source = preprocess(source)
    target = preprocess(target)

  source_tokens = source.split() #List of tokens: ['X', 'Y',...]
  target_tokens = target.split()

  vocab_set = set(source_tokens + target_tokens)

  mutual_vocab_dict = {}
  for token in vocab_set:
    if (token in source_tokens) and (token in target_tokens):
      mutual_vocab_dict[token] = 1
    else:
      mutual_vocab_dict[token] = 0

  source_mutual_ids = [mutual_vocab_dict[token] for token in source_tokens] # List of match value (0: not match, 1:match) [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1]
  target_mutual_ids = [mutual_vocab_dict[token] for token in target_tokens]

  src_aligned_index = generate_aligned_index(source_mutual_ids, len(source_mutual_ids)) #Generate aligned index by match token between 2 sentences
  tgt_aligned_index = generate_aligned_index(target_mutual_ids, len(target_mutual_ids)) #[0, 1, 2, 2, 3, 4, 4, 4, 4, 5, 6, 7]

  src_missing_intervals = find_missing_intervals(source_mutual_ids) #Interval index mising: [[0], [2, 3], [5, 6, 7, 8], [10]]
  tgt_missing_intervals = find_missing_intervals(target_mutual_ids)

  first_src_aligned_tokens = align_tokens(src_aligned_index, source_tokens) #Align tokens by aligned index
  first_tgt_aligned_tokens = align_tokens(tgt_aligned_index, target_tokens)
  #print(first_src_aligned_tokens)
  #print(first_tgt_aligned_tokens)

  src_aligned_tokens, tgt_aligned_tokens = check_realign(first_src_aligned_tokens, first_tgt_aligned_tokens)


  return source_tokens, src_aligned_tokens, target_tokens, tgt_aligned_tokens

In [ ]:
source = ViLexNorm.iloc[8726]['original']
target = ViLexNorm.iloc[8726]['normalized']

source_tokens, src_aligned_tokens, target_tokens, tgt_aligned_tokens = align(source, target, segment=False)

In [ ]:
print(source_tokens)
print(target_tokens)

print("")
print(src_aligned_tokens)
print(tgt_aligned_tokens)


['cơm', 'tróa', 'quá', 'tr', 'oi']
['cơm', 'chó', 'quá', 'trời', 'ơi']

[['cơm'], ['tróa'], ['quá'], ['tr'], ['oi']]
[['cơm'], ['chó'], ['quá'], ['trời'], ['ơi']]


# **Post-processing and Create NSW dictionary**

# Word-level

## Preprocess annotated data

In [ ]:
def get_pair_df(data):
  flat_data = []
  for sublist_a, sublist_b in zip(data[0], data[1]):
    if len(sublist_a) > 1:
        sublist_a = [' '.join(sublist_a)]
    if len(sublist_b) > 1:
        sublist_b = [' '.join(sublist_b)]
    for a, b in zip(sublist_a, sublist_b):
        flat_data.append((a, b))

  # Create a DataFrame from the flattened data
  df = pd.DataFrame(flat_data, columns=['token', 'norm'])
  return df

In [ ]:
LexNorm_word = []
for i in tqdm (range(len(ViLexNorm)), desc="Loading..."):
  source = ViLexNorm['original'][i]
  target = ViLexNorm['normalized'][i]
  source_tokens, src_aligned_tokens, target_tokens, tgt_aligned_tokens = align(source, target, segment=True)
  LexNorm_word.append([src_aligned_tokens, tgt_aligned_tokens])

Loading...: 100%|██████████| 10467/10467 [03:09<00:00, 55.10it/s]


In [ ]:
word_normalized = pd.DataFrame()
for i in tqdm (range(len(LexNorm_word)), desc="Loading..."):
  data = LexNorm_word[i]
  x= get_pair_df(data)
  sentence = "Index: " + str(i)
  word_normalized = pd.concat([word_normalized,pd.DataFrame(data = {'index': [sentence], 0: np.nan})])
  word_normalized = pd.concat([word_normalized, x.reset_index(drop=False)])

word_normalized.columns = ['sent_idx', 'token_idx', 'token','norm']
word_normalized['token_idx'] = word_normalized['sent_idx']

sent_idx = -1
for row in range(len(word_normalized)):
    if('Index' in str(word_normalized.iloc[row, 0])):
      word_normalized.iloc[row, 0] = np.nan
      word_normalized.iloc[row, 1] = np.nan
      sent_idx += 1
    else:
      word_normalized.iloc[row, 0] = sent_idx
word_normalized = word_normalized.dropna(subset = ['token_idx', 'token', 'norm'], how='all')
word_normalized = word_normalized.reset_index(drop=True)
word_normalized['token_idx'] = word_normalized['token_idx'].astype(int)


Loading...: 100%|██████████| 10467/10467 [02:51<00:00, 61.11it/s]


In [ ]:
word_normalized

,sent_idx,token_idx,token,norm
0,0,0,thích,thích
1,0,1,anh,anh
2,0,2,cá_mập,cá_mập
3,0,3,k,không
4,1,0,cứ,cứ
...,...,...,...,...
120542,10466,0,bí_ẩn,bí_ẩn
120543,10466,1,j,gì
120544,10466,2,v,vậy
120545,10466,3,mn,mọi người


In [ ]:
#word_normalized.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/LexNorm_word.csv", index=False)

## Post_processing ViLexNorm

In [164]:
path = '/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/raw_data/word_dict/LexNorm_word(checked).csv'

In [168]:
data = pd.read_csv(path, keep_default_na=False)
data['label'] = data['token'] != data['norm']
# label: 0 (standard), 1 (non-standard)
data

,sent_idx,token_idx,token,norm,label
0,0,0,thích,thích,False
1,0,1,anh,anh,False
2,0,2,cá_mập,cá_mập,False
3,0,3,k,không,True
4,1,0,cứ,cứ,False
...,...,...,...,...,...
121097,10466,0,bí_ẩn,bí_ẩn,False
121098,10466,1,j,gì,True
121099,10466,2,v,vậy,True
121100,10466,3,mn,mọi người,True


In [171]:
data[data.sent_idx == 434]

,sent_idx,token_idx,token,norm,label
5239,434,0,bạn,bạn,False
5240,434,1,biết_điều,biết_điều,False
5241,434,2,tệ,tệ,False
5242,434,3,nhất,nhất,False
5243,434,4,trong,trong,False
5244,434,5,câu_chuyện,câu_chuyện,False
5245,434,6,này,này,False
5246,434,7,ko,không,True
5247,434,8,?,?,False


In [172]:
std_df = data[['sent_idx', 'label']].groupby(['sent_idx']).sum()
std_idx = std_df.loc[std_df['label'] == 0].index
std_idx

Index([433, 4904, 5334, 8435], dtype='int64', name='sent_idx')

In [173]:
data = data[~data['sent_idx'].isin(std_idx)].reset_index(drop = True)
data

,sent_idx,token_idx,token,norm,label
0,0,0,thích,thích,False
1,0,1,anh,anh,False
2,0,2,cá_mập,cá_mập,False
3,0,3,k,không,True
4,1,0,cứ,cứ,False
...,...,...,...,...,...
121067,10466,0,bí_ẩn,bí_ẩn,False
121068,10466,1,j,gì,True
121069,10466,2,v,vậy,True
121070,10466,3,mn,mọi người,True


In [179]:
new_data = []
i=0
for sent_idx in data.groupby(['sent_idx']).sum().index:
  sentence = {}
  sentence['id'] = i
  sentence['input'] = list(data.loc[data['sent_idx'] == sent_idx]['token'])
  sentence['output'] = list(data.loc[data['sent_idx'] == sent_idx]['norm'])
  new_data.append(sentence)
  i+=1

In [181]:
train_json = new_data[:8369]
test_json = new_data[8370:9412]
dev_json = new_data[9413:]

In [183]:
#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/Token_data/Segment/ViLexNorm_segment.json", "w") as outfile:
    #json.dump(new_data, outfile, ensure_ascii=False)

#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/Token_data/Segment/train_segment.json", "w") as outfile:
    #json.dump(train_json, outfile, ensure_ascii=False)

#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/Token_data/Segment/test_segment.json", "w") as outfile:
    #json.dump(test_json, outfile, ensure_ascii=False)

#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/Token_data/Segment/dev_segment.json", "w") as outfile:
    #json.dump(dev_json, outfile, ensure_ascii=False)

In [ ]:
ViLexNorm = []
for sent_idx in data.groupby(['sent_idx']).sum().index:
  sentence = {}
  sentence['id'] = sent_idx
  sentence['input'] = ' '.join(list(data.loc[data['sent_idx'] == sent_idx]['token'])).replace('_', ' ')
  sentence['output'] = ' '.join(list(data.loc[data['sent_idx'] == sent_idx]['norm'])).replace('_', ' ')
  ViLexNorm.append(sentence)
ViLexNorm = pd.DataFrame(ViLexNorm)
ViLexNorm = ViLexNorm.drop(columns=['id'])
ViLexNorm.columns = ['original', 'normalized']
ViLexNorm

<ipython-input-41-7e00faebbc3b>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  for sent_idx in data.groupby(['sent_idx']).sum().index:


,original,normalized
0,thích anh cá mập k,thích anh cá mập không
1,cứ ngây thơ thế thoai :)),cứ ngây thơ thế thôi :))
2,bà nghê xinh vậy mà t thấy k bằng bà chipu luô...,bà nghê xinh vậy mà tôi thấy không bằng bà chi...
3,ê k khóc được làm thế nào má =)) ?,ê không khóc được làm thế nào má =)) ?
4,có biến gì hong dẫy :)),có biến gì không vậy :))
...,...,...
10458,từ lúc đu idol hàn và diễn viên thái thì hầu n...,từ lúc đu idol hàn và diễn viên thái thì hầu n...
10459,a hay nói e á,anh hay nói em á
10460,đọc jd mà nó dễ quá đâm ra sợ cty lừa :))),đọc jd mà nó dễ quá đâm ra sợ công ty lừa :)))
10461,chỗ còn tuyển hong rủ tui vô làm nói chuyện dí...,chỗ còn tuyển không rủ tui vô làm nói chuyện v...


In [ ]:
train = ViLexNorm[:8369]
test = ViLexNorm[8370:9412]
dev = ViLexNorm[9413:]
print(len(train), len(test), len(dev))

8369 1042 1050


In [ ]:
#train.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/train.csv", index=False)
#test.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/test.csv", index=False)
#dev.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/dev.csv", index=False)
#ViLexNorm.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/ViLexNorm.csv", index=False)

## Create LexNorm Dictionary

In [ ]:
def count_vocab(data):
  vocab = []
  for sent in data:
    sent = sent['input']
    for token in sent:
      if (re.search('([.,!?()“”;"\':…])', token) is None) & (token != ''):
        vocab.append(token)

  return len(vocab), len(set(vocab))

In [ ]:
count_vocab(new_data)

(114759, 10734)

In [ ]:
def nsw(data):
  nsw_lst = []
  for sent in data:
    inp = sent['input']
    out = sent['output']
    for i in range(len(inp)):
      if((inp[i] != '') & (inp[i] != out[i])):
        nsw_lst.append(inp[i])

  return(nsw_lst)

In [ ]:
new_data[0]

{'id': 0,
 'input': ['thích', 'anh', 'cá_mập', 'k'],
 'output': ['thích', 'anh', 'cá_mập', 'không']}

In [ ]:
LexNorm_dict_word = {}
for i in tqdm (range(len(new_data)), desc="Loading..."):
  inp = new_data[i].get('input')
  out = new_data[i].get('output')
  for i in range(len(inp)):
    if((inp[i] != '') & (inp[i] != out[i])):
      key = inp[i]
      value = out[i]
      key_str = ''.join(key)
      value_str = ''.join(value)
      if key_str not in LexNorm_dict_word:
        LexNorm_dict_word[key_str] = []
      if value_str not in LexNorm_dict_word[key_str]:
        LexNorm_dict_word[key_str].append(value_str)

Loading...: 100%|██████████| 10463/10463 [00:00<00:00, 105824.51it/s]


In [ ]:
#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/LexNorm_dict_segment.json", "w") as outfile:
    #json.dump(LexNorm_dict_word, outfile, ensure_ascii=False)

In [ ]:
# prompt: convert dictionary LexNorm_dict_word to DataFrame
LexNorm_dict_word_df = pd.DataFrame(LexNorm_dict_word.items(), columns=["Original", "Normalized"])
LexNorm_dict_word_df

,Original,Normalized
0,k,[không]
1,thoai,[thôi]
2,t,"[tôi, thằng, tao, tớ, tiếng, tui, tồi, ta, tới..."
3,chời,[trời]
4,hong,"[không, hông, hóng]"
...,...,...
3648,saooooo,[sao]
3649,ngok ngheck,[ngốc_nghếch]
3650,người eo,[người_yêu]
3651,họnc,[học]


# Syllable-level

## Preprocess annotated data

In [ ]:
LexNorm_syllable = []
for i in tqdm (range(len(ViLexNorm)), desc="Loading..."):
  source = ViLexNorm['original'][i]
  target = ViLexNorm['normalized'][i]
  source_tokens, src_aligned_tokens, target_tokens, tgt_aligned_tokens = align(source, target, segment=False)
  LexNorm_syllable.append([src_aligned_tokens, tgt_aligned_tokens])

Loading...: 100%|██████████| 10463/10463 [01:07<00:00, 155.17it/s]


In [ ]:
syllable_normalized = pd.DataFrame()
for i in tqdm (range(len(LexNorm_syllable)), desc="Loading..."):
  data = LexNorm_syllable[i]
  x= get_pair_df(data)
  sentence = "Index: " + str(i)
  syllable_normalized = pd.concat([syllable_normalized,pd.DataFrame(data = {'index': [sentence], 0: np.nan})])
  syllable_normalized = pd.concat([syllable_normalized, x.reset_index(drop=False)])

syllable_normalized.columns = ['sent_idx', 'token_idx', 'token','norm']
syllable_normalized['token_idx'] = syllable_normalized['sent_idx']

sent_idx = -1
for row in range(len(syllable_normalized)):
    if('Index' in str(syllable_normalized.iloc[row, 0])):
      syllable_normalized.iloc[row, 0] = np.nan
      syllable_normalized.iloc[row, 1] = np.nan
      sent_idx += 1
    else:
      syllable_normalized.iloc[row, 0] = sent_idx
syllable_normalized = syllable_normalized.dropna(subset = ['token_idx', 'token', 'norm'], how='all')
syllable_normalized = syllable_normalized.reset_index(drop=True)
syllable_normalized['token_idx'] = syllable_normalized['token_idx'].astype(int)


Loading...: 100%|██████████| 10463/10463 [01:28<00:00, 118.29it/s]


In [ ]:
syllable_normalized

,sent_idx,token_idx,token,norm
0,0,0,thích,thích
1,0,1,anh,anh
2,0,2,cá,cá
3,0,3,mập,mập
4,0,4,k,không
...,...,...,...,...
132769,10462,1,ẩn,ẩn
132770,10462,2,j,gì
132771,10462,3,v,vậy
132772,10462,4,mn,mọi người


In [ ]:
#syllable_normalized.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/syllable_dict/LexNorm_syllable.csv", index=False)

## Post_processing ViLexNorm

In [146]:
path = '/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/raw_data/syllable_dict/LexNorm_syllable(checked).csv'

In [147]:
data = pd.read_csv(path, keep_default_na=False)
data['label'] = data['token'] != data['norm']
# label: 0 (standard), 1 (non-standard)
data

,sent_idx,token_idx,token,norm,label
0,0,0,thích,thích,False
1,0,1,anh,anh,False
2,0,2,cá,cá,False
3,0,3,mập,mập,False
4,0,4,k,không,True
...,...,...,...,...,...
133334,10462,1,ẩn,ẩn,False
133335,10462,2,j,gì,True
133336,10462,3,v,vậy,True
133337,10462,4,mn,mọi người,True


In [148]:
std_df = data[['sent_idx', 'label']].groupby(['sent_idx']).sum()
std_idx = std_df.loc[std_df['label'] == 0].index
std_idx

Index([], dtype='int64', name='sent_idx')

In [149]:
data = data[~data['sent_idx'].isin(std_idx)].reset_index(drop = True)
data

,sent_idx,token_idx,token,norm,label
0,0,0,thích,thích,False
1,0,1,anh,anh,False
2,0,2,cá,cá,False
3,0,3,mập,mập,False
4,0,4,k,không,True
...,...,...,...,...,...
133334,10462,1,ẩn,ẩn,False
133335,10462,2,j,gì,True
133336,10462,3,v,vậy,True
133337,10462,4,mn,mọi người,True


In [150]:
new_data = []
for sent_idx in data.groupby(['sent_idx']).sum().index:
  sentence = {}
  sentence['id'] = sent_idx
  sentence['input'] = list(data.loc[data['sent_idx'] == sent_idx]['token'])
  sentence['output'] = list(data.loc[data['sent_idx'] == sent_idx]['norm'])
  new_data.append(sentence)

In [151]:
train_json = new_data[:8369]
test_json = new_data[8370:9412]
dev_json = new_data[9413:]

In [152]:
#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/Token_data/Syllable/ViLexNorm_syllable.json", "w") as outfile:
    #json.dump(new_data, outfile, ensure_ascii=False)

#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/Token_data/Syllable/train_syllable.json", "w") as outfile:
    #json.dump(train_json, outfile, ensure_ascii=False)

#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/Token_data/Syllable/test_syllable.json", "w") as outfile:
    #json.dump(test_json, outfile, ensure_ascii=False)

#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/Token_data/Syllable/dev_syllable.json", "w") as outfile:
    #json.dump(dev_json, outfile, ensure_ascii=False)

In [153]:
ViLexNorm = []
for sent_idx in range(0, 10463):
  sentence = {}
  sentence['id'] = sent_idx
  sentence['input'] = ' '.join(list(data.loc[data['sent_idx'] == sent_idx]['token']))
  sentence['output'] = ' '.join(list(data.loc[data['sent_idx'] == sent_idx]['norm']))
  ViLexNorm.append(sentence)
ViLexNorm = pd.DataFrame(ViLexNorm)

ViLexNorm = ViLexNorm.drop(columns=['id'])
ViLexNorm.columns = ['original', 'normalized']
ViLexNorm

,original,normalized
0,thích anh cá mập k,thích anh cá mập không
1,cứ ngây thơ thế thoai :)),cứ ngây thơ thế thôi :))
2,bà nghê xinh vậy mà t thấy k bằng bà chipu luô...,bà nghê xinh vậy mà tôi thấy không bằng bà chi...
3,ê k khóc được làm thế nào má =)) ?,ê không khóc được làm thế nào má =)) ?
4,có biến gì hong dẫy :)),có biến gì không vậy :))
...,...,...
10458,từ lúc đu idol hàn và diễn viên thái thì hầu n...,từ lúc đu idol hàn và diễn viên thái thì hầu n...
10459,a hay nói e á,anh hay nói em á
10460,đọc jd mà nó dễ quá đâm ra sợ cty lừa :))),đọc jd mà nó dễ quá đâm ra sợ công ty lừa :)))
10461,chỗ còn tuyển hong rủ tui vô làm nói chuyện dí...,chỗ còn tuyển không rủ tui vô làm nói chuyện v...


In [154]:
train = ViLexNorm[:8369]
test = ViLexNorm[8370:9412]
dev = ViLexNorm[9413:]
print(len(train), len(test), len(dev))

8369 1042 1050


In [155]:
#train.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/train.csv", index=False)
#test.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/test.csv", index=False)
#dev.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/dev.csv", index=False)
#ViLexNorm.to_csv("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/ViLexNorm.csv", index=False)

## Create LexNorm Dictionary

In [156]:
def count_vocab(data):
  vocab = []
  for sent in data:
    sent = sent['input']
    for token in sent:
      if (re.search('([.,!?()“”;"\':…])', token) is None) & (token != ''):
        vocab.append(token)

  return len(vocab), len(set(vocab))

In [157]:
count_vocab(new_data)

(127035, 7228)

In [158]:
def nsw(data):
  nsw_lst = []
  for sent in data:
    inp = sent['input']
    out = sent['output']
    for i in range(len(inp)):
      if((inp[i] != '') & (inp[i] != out[i])):
        nsw_lst.append(inp[i])

  return(nsw_lst)

In [159]:
new_data[0]

{'id': 0,
 'input': ['thích', 'anh', 'cá', 'mập', 'k'],
 'output': ['thích', 'anh', 'cá', 'mập', 'không']}

In [160]:
LexNorm_dict_syllable = {}
for i in tqdm (range(len(new_data)), desc="Loading..."):
  inp = new_data[i].get('input')
  out = new_data[i].get('output')
  for i in range(len(inp)):
    if((inp[i] != '') & (inp[i] != out[i])):
      key = inp[i]
      value = out[i]
      key_str = ''.join(key)
      value_str = ''.join(value)
      if key_str not in LexNorm_dict_syllable:
        LexNorm_dict_syllable[key_str] = []
      if value_str not in LexNorm_dict_syllable[key_str]:
        LexNorm_dict_syllable[key_str].append(value_str)

Loading...: 100%|██████████| 10463/10463 [00:00<00:00, 57186.31it/s]


In [163]:
#with open("/content/drive/MyDrive/LexNorm/KLTN/Data/ViLexNorm/LexNorm_dict_syllable.json", "w") as outfile:
    #json.dump(LexNorm_dict_syllable, outfile, ensure_ascii=False)

In [162]:
# prompt: convert dictionary LexNorm_dict_word to DataFrame
LexNorm_dict_syllable_df = pd.DataFrame(LexNorm_dict_syllable.items(), columns=["Original", "Normalized"])
LexNorm_dict_syllable_df

,Original,Normalized
0,k,[không]
1,thoai,[thôi]
2,t,"[tôi, thằng, tao, tớ, tiếng, tui, tồi, ta, tới..."
3,chời,[trời]
4,hong,"[không, hông, hóng]"
...,...,...
3086,đýt,[đít]
3087,pói,[bói]
3088,saooooo,[sao]
3089,ngheck,[nghếch]
